## Análisis de sentimientos de Twitter con Python
* https://platzi.com/tutoriales/1874-python-lenguaje-natural/5654-realiza-un-analisis-de-sentimiento-en-3-pasos-con-python/
    * Ejemplo https://twitter.com/whaleandjaguar_?lang=en
* https://towardsdatascience.com/my-absolute-go-to-for-sentiment-analysis-textblob-3ac3a11d524
* https://www.justintodata.com/twitter-sentiment-analysis-python/
* https://www.pluralsight.com/guides/building-a-twitter-sentiment-analysis-in-python

### 1) Descargar tweets

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
pd.set_option('display.max_colwidth',500)

In [31]:
df=pd.read_json('https://raw.githubusercontent.com/restrepo/twitter/main/tweets_df.json')

In [32]:
df=df.append(df1).reset_index(drop=True)

In [33]:
df.shape

(10000, 10)

In [35]:
df.drop_duplicates(subset='text').shape

(2384, 10)

### Cree un nuevo clasificador
Vamos 
https://textblob.readthedocs.io/en/dev/classifiers.html#classifiers

* A favor del Paro → `'fav'`
* En contra del Paro→ `'con'`
* Informativo or neutro → `'neu'`
* Spam → `'spa'`

Para crear los conjutos de datos para training y para test debemos clasisificar un conjunto suficientemente grande de textos de tweets en las dos categorias en números similares para cada una de las dos categorias

Crearemos un algoritmo que nos permite clasificar algunos tweets del DataFrame en una nueva columna `'label'`, la cual tendrá un valor nulo cuando el tweet no este clasificado. Una vez un tweet sea clasificado, se añadirá a una lista de similaridad y sólo se clasificarán nuevos tweets que no sean similares a los previamente clasificados.  Para ello se usará el método `extractOne` del módulo `process` de `fuzzywuzzy` con scorer `fuzz.ratio` basado en la distancia Levenshtein entre dos textos.

In [5]:
from fuzzywuzzy import process
from fuzzywuzzy import  fuzz

In [ ]:
n_train=100
n_test=50
n_traintest=n_train+n_test
df['label']=None
ii=0
similarity=[]
for i in df.index:
    print("="*80)
    tweet=df.loc[i,'text']
    if similarity:
        if process.extractOne(tweet,similarity,scorer=fuzz.ratio)[-1]<90:
            similarity.append(tweet)
            posneg=input(f"{tweet}: https://twitter.com/twitter/status/{df.loc[i,'id']}\n → f/c/n/s para 'a favor'/'en contra'/'neutro'/'spam' or <Enter> para continuar:\n")
            if posneg=='c':
                posneg='con'
            elif posneg=='f':
                posneg='fav'
            elif posneg=='n':
                posneg='neu'
            elif posneg=='s':
                posneg='spa'
            else:
                continue
            df.loc[i,'label']=posneg
            ii=ii+1
    else:
        similarity.append(tweet)

    if ii==n_traintest:
        break

Select the train/test filtered DataFrame

In [7]:
tmp=df[~df['label'].isna()].reset_index(drop=True)

In [8]:
tmp.shape

(143, 11)

In [9]:
tmp[:3]

,user_name,user_location,user_description,user_verified,date,text,hashtags,source,id,original_id,label
0,Sandra Suarez - opiniones personales,,,False,2021-05-22 10:16:48,"RT @CapitnColombia: Por su presente, por su futuro, Construiremos un nuevo país. PRIMERA LINEA Y CAPITAN COLOMBIA!. \n[ 📸Diego Cortes ]\n#par…",None,Twitter for Android,1396047383784529920,1.395886e+18,fav
1,@marioperico4,,abajotwitter,False,2021-05-22 10:16:41,RT @martinquinco: @jflafaurie @UNPColombia Desbloqueo de vías ya!\n#NoMasParo \n#NoMasBloqueos \n#YoApoyoALaFuerzaPublica,"[NoMasParo, NoMasBloqueos, YoApoyoALaFuerzaPublica]",Twitter for Android,1396047356123041792,1.395756e+18,con
2,Mauro Gabriel Escalona Quiroz,,"Antiuribista 100% y 24/7, por que Colombia merece vivir en paz. Venceremos al tirano!!",False,2021-05-22 10:16:36,RT @Contagioradio1: #Bogotá Chicos de la guardia comunitaria primera línea en el portal de la resistencia protegiendo a la comunidad #ParoN…,[Bogotá],Twitter for Android,1396047336439259136,1.395937e+18,fav


In [10]:
tmp.shape

(143, 11)

In [11]:
train=tmp[:n_train][['text','label']].reset_index(drop=True)

In [12]:
from textblob.classifiers import NaiveBayesClassifier

In [13]:
import nltk
nltk.download('punkt')  

[nltk_data] Downloading package punkt to /home/restrepo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
cl = NaiveBayesClassifier(  
    [ (d.get('text'),d.get('label')) for d in train.to_dict(orient='records')]  )

In [15]:
test=tmp[n_train:][['text','label']].reset_index(drop=True)

In [24]:
train.to_json('train.json',orient='records')
test.to_json('train.json',orient='records')

In [16]:
fulltest=test.copy()
fulltest['test']=fulltest['text'].apply(cl.classify)

In [18]:
fulltest[['text','label','test']][:3]

,text,label,test
0,Grupo de WhatsApp: cambio y ventas de cuentas frefire COMPRAR Y VENDER CUENTAS DE FREFIRE... | #Cali #Amigos cali #Amigas cali #Amistad cali #Conocer gente cali #Gamers #Games #Videojuegos #Juegos #Jugadores #Gaming #Juegos online #Jugar online #Ps4 #P... https://t.co/GPwjZ42dW7,spa,con
1,"RT @FRANCISCOLEALM: Bueno, ya‼️\nTodo esto era un paro‼️\nYa‼️ no más... \n#ParoNacional21M #NoMasParo #CIDHEnColombia #BloqueoMental \n@gusgom…",con,con
2,"RT @Elolfato: #ParoNacional | ""Hay infiltrados que aprovechan las manifestaciones para cometer actos delictivos"": CGT\n\n@CGTCol \n\nMás detall…",con,con


In [19]:
cl.accuracy( [ (d.get('text'),d.get('label')) for d in test.to_dict(orient='records')] )

0.7441860465116279

In [20]:
fulltest['prob']=fulltest['text'].apply(lambda t:   
                                cl.prob_classify( t ).prob(   cl.classify( t  )  ) ).round(2)

In [22]:
fulltest[:3]

,text,label,test,prob
0,Grupo de WhatsApp: cambio y ventas de cuentas frefire COMPRAR Y VENDER CUENTAS DE FREFIRE... | #Cali #Amigos cali #Amigas cali #Amistad cali #Conocer gente cali #Gamers #Games #Videojuegos #Juegos #Jugadores #Gaming #Juegos online #Jugar online #Ps4 #P... https://t.co/GPwjZ42dW7,spa,con,0.91
1,"RT @FRANCISCOLEALM: Bueno, ya‼️\nTodo esto era un paro‼️\nYa‼️ no más... \n#ParoNacional21M #NoMasParo #CIDHEnColombia #BloqueoMental \n@gusgom…",con,con,0.98
2,"RT @Elolfato: #ParoNacional | ""Hay infiltrados que aprovechan las manifestaciones para cometer actos delictivos"": CGT\n\n@CGTCol \n\nMás detall…",con,con,0.96


Aplicar al DataFrame Completo

In [36]:
df['test']=df['text'].apply(cl.classify)
df['prob']=df['text'].apply(lambda t:   
                                cl.prob_classify( t ).prob(   cl.classify( t  )  ) ).round(2)

In [37]:
dfp=df.drop_duplicates('text').reset_index(drop=True)

In [38]:
dfp[['text','test','prob']].sample(10)

,text,test,prob
1408,"#ParoNacional\n#1deMayo\n#EsmadAsesinos\n#ReformaTributaria\n#ParoNacional1M\n#DuquePareLaViolencia\n#Bogota\n#UribeDioLaOrden\n#SOSColombiaDDHH\n#ATENCION\n#DDHH\n Bogotá, Colombia. Paro Nacional - Asamblea informativa de maestros I.E.D... https://t.co/mQalO9k2xQ a través de @YouTube",fav,0.65
689,RT @teleSURtv: #Colombia 🇨🇴 | Ciudadanos denunciaron este martes que el #Esmad de la Policía reprimió la noche del lunes a las personas que…,fav,0.59
1954,@IvanCepedaCast El derecho a la protesta entierra los demás derechos ciudadanos en #Colombia! El pueblo es sometido por la dictadura de bloqueos y vandalismo! #UribeTieneLaRazon #PetroNuncaSeraPresidente #PetroSeHaceODIAR,fav,0.80
1157,#Buenaventura #BuenaventuraResiste #SOSBuenaventura \nEXPLOSIÓN PETARDOS \nRESISTENCIA \n#UribeDioLaOrden #ElParoNOPara\n #ColombiaResiste #ColombiaSOS #SOSOCOLOMBIA #PrimeraLinea #GraciasPrimeraLinea #SOSColombiaDDDHH #ParoNacional17M #ParoNacional https://t.co/3AA00H1OVk,con,0.65
71,RT @MafeCarrascal: ¿Y mañana sábado?¿Monumento a Los Héroes en Bogotá? #ParoNacional,con,0.93
2255,@petrogustavo #NoMasParo #PetroEsUnHampon #PETRO DESTRUYE A COLOMBIA https://t.co/dpc2tZCder,con,1.00
1678,TRAIGA MENOR QUE SE LLEVARON AYER \nPIDA PERDON X LAS VIOLACIONES A LA MUJER \n#PortalAmericas RESISTE SIN NEGOCIACIONES #Colombia #Bogotá #bogotáenalertaroja #BogotáResiste\n#PrimeraLinea #GraciasPrimeraLinea #ParoNacional18M #ParoNacional18M #ParoNacional #ElParoNOPara https://t.co/4BR1i4KYAa,con,0.83
514,"#Rusia rechaza declaraciones del Ministro de Defensa, Diego Molano de #Colombia, quien aseguró que, en un ejercicio de ciberseguridad, durante las protestas han salido 102 noticias falsas para desacreditar a la policía. #ParoNacional\n\nhttps://t.co/1S3fAuOnOi",fav,0.99
1701,Se me acaban las esperanzas... #NoMasParo #NoMasBloqueos #Rechazo #VandalismoPuroYDuro https://t.co/QCvB3o61Ar,con,1.00
104,"RT @CesarJerezM: Esta mañana fue encontrada la cabeza de un muchacho en un contenedor de basura del Colegio Alfonso López, cerca al @almac…",fav,0.97


In [39]:
dfp.groupby('test')['test'].count()

test
con    1193
fav    1191
Name: test, dtype: int64

Sin quitar RT

In [40]:
df.groupby('test')['test'].count()

test
con    4008
fav    5992
Name: test, dtype: int64